In [ ]:
!nvidia-smi
import tensorflow as tf
print("GPU visível p/ TF:", tf.config.list_physical_devices("GPU"))


In [1]:
import requests, zipfile
from pathlib import Path

URL = "https://download.microsoft.com/download/3/e/1/3e1c3f21-ecdb-4869-8368-6deba77b919f/kagglecatsanddogs_5340.zip"
ZIP = Path("/content/catsdogs.zip")
RAW = Path("/content/catsdogs_raw")

if not ZIP.exists():
    print("Baixando…")
    with requests.get(URL, headers={"User-Agent":"Mozilla/5.0"}, stream=True) as r:
        r.raise_for_status()
        with open(ZIP, "wb") as f:
            for chunk in r.iter_content(1024*1024):
                if chunk: f.write(chunk)

if not RAW.exists():
    print("Extraindo…")
    with zipfile.ZipFile(ZIP, "r") as z:
        z.extractall(RAW)

print("OK:", RAW.exists())


Baixando…
Extraindo…
OK: True


In [2]:
from pathlib import Path
from PIL import Image
import random, shutil

SRC  = Path("/content/catsdogs_raw/PetImages")
ROOT = Path("/content/data")
for s in ("train","val","test"):
    for c in ("Cat","Dog"):
        (ROOT/s/c).mkdir(parents=True, exist_ok=True)

def is_ok(p: Path):
    try:
        with Image.open(p) as im: im.verify()
        return True
    except: return False

def split_copy(cls: str):
    files = list((SRC/cls).glob("*.jpg"))
    files = [f for f in files if f.stat().st_size > 0 and is_ok(f)]
    random.seed(42); random.shuffle(files)
    n = len(files); n_tr = int(0.70*n); n_val = int(0.15*n)
    tr, va, te = files[:n_tr], files[n_tr:n_tr+n_val], files[n_tr+n_val:]
    for f in tr: shutil.copy2(f, ROOT/"train"/cls/f.name)
    for f in va: shutil.copy2(f, ROOT/"val"/cls/f.name)
    for f in te: shutil.copy2(f, ROOT/"test"/cls/f.name)
    return len(tr), len(va), len(te)

print("Cat:", split_copy("Cat"))
print("Dog:", split_copy("Dog"))
print("Base:", ROOT)


Cat: (8749, 1874, 1876)


/usr/local/lib/python3.12/dist-packages/PIL/TiffImagePlugin.py:950: UserWarning: Truncated File Read
  warnings.warn(str(msg))


Dog: (8749, 1874, 1876)
Base: /content/data


In [3]:
import tensorflow as tf
from tensorflow.keras.applications.vgg16 import preprocess_input

IMG_SIZE = 224
BATCH = 32  # se a GPU lotar, reduza para 16

def _parse(path: tf.Tensor):
    img = tf.io.read_file(path)
    img = tf.image.decode_jpeg(img, channels=3)   # força 3 canais
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])
    img = tf.cast(img, tf.float32)
    img = preprocess_input(img)
    is_cat = tf.strings.regex_full_match(path, ".*/Cat/.*")
    label = tf.where(is_cat, 0, 1)
    label = tf.one_hot(label, 2)
    return img, label

def make_ds(pattern, shuffle=True):
    ds = tf.data.Dataset.list_files(pattern, shuffle=shuffle)
    if shuffle:
        ds = ds.shuffle(50_000, reshuffle_each_iteration=True)
    ds = ds.map(_parse, num_parallel_calls=tf.data.AUTOTUNE)
    ds = ds.apply(tf.data.experimental.ignore_errors())
    ds = ds.batch(BATCH).prefetch(tf.data.AUTOTUNE)
    return ds

train = make_ds("/content/data/train/*/*.jpg", shuffle=True)
val   = make_ds("/content/data/val/*/*.jpg",   shuffle=False)
test  = make_ds("/content/data/test/*/*.jpg",  shuffle=False)

xb, yb = next(iter(train))
print("Batch:", xb.shape, yb.shape)  # ex.: (32, 224, 224, 3) (32, 2)


Instructions for updating:
Use `tf.data.Dataset.ignore_errors` instead.


Batch: (32, 224, 224, 3) (32, 2)


In [4]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np

# Base pré-treinada
base = keras.applications.VGG16(include_top=False, weights="imagenet",
                                input_shape=(224,224,3))
base.trainable = False

inputs = keras.Input(shape=(224,224,3))
x = base(inputs, training=False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
outputs = layers.Dense(2, activation="softmax")(x)
model = keras.Model(inputs, outputs)

model.compile(optimizer=keras.optimizers.Adam(1e-3),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

callbacks = [
    keras.callbacks.EarlyStopping(patience=2, restore_best_weights=True),
    keras.callbacks.ReduceLROnPlateau(patience=1, factor=0.5, verbose=1),
]

print("Treinando (base congelada)…")
history = model.fit(train, validation_data=val, epochs=5, callbacks=callbacks)

print("Teste (congelada):")
model.evaluate(test, verbose=2)

# Fine-tuning: liberar só o bloco 5 da VGG16
for layer in base.layers:
    layer.trainable = layer.name.startswith("block5")

model.compile(optimizer=keras.optimizers.Adam(1e-4),
              loss="categorical_crossentropy",
              metrics=["accuracy"])

print("Treinando (fine-tuning)…")
history_ft = model.fit(train, validation_data=val, epochs=3, callbacks=callbacks)

print("Teste (fine-tuning):")
model.evaluate(test, verbose=2)

# Relatório final
y_true, y_pred = [], []
for xb, yb in test:
    pr = model.predict(xb, verbose=0)
    y_true.extend(np.argmax(yb.numpy(), axis=1))
    y_pred.extend(np.argmax(pr, axis=1))

print(classification_report(y_true, y_pred, target_names=["Cat","Dog"]))
print("Matriz de confusão:\n", confusion_matrix(y_true, y_pred))


58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Treinando (base congelada)…
Epoch 1/5
    543/Unknown 124s 200ms/step - accuracy: 0.8526 - loss: 1.2011

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


543/543 ━━━━━━━━━━━━━━━━━━━━ 150s 247ms/step - accuracy: 0.8527 - loss: 1.1998 - val_accuracy: 0.9796 - val_loss: 0.0976 - learning_rate: 0.0010
Epoch 2/5
543/543 ━━━━━━━━━━━━━━━━━━━━ 116s 214ms/step - accuracy: 0.9708 - loss: 0.1314 - val_accuracy: 0.9820 - val_loss: 0.0770 - learning_rate: 0.0010
Epoch 3/5
543/543 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.9784 - loss: 0.0829
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
543/543 ━━━━━━━━━━━━━━━━━━━━ 116s 214ms/step - accuracy: 0.9784 - loss: 0.0829 - val_accuracy: 0.9804 - val_loss: 0.0780 - learning_rate: 0.0010
Epoch 4/5
543/543 ━━━━━━━━━━━━━━━━━━━━ 116s 213ms/step - accuracy: 0.9805 - loss: 0.0716 - val_accuracy: 0.9839 - val_loss: 0.0577 - learning_rate: 5.0000e-04
Epoch 5/5
543/543 ━━━━━━━━━━━━━━━━━━━━ 0s 176ms/step - accuracy: 0.9820 - loss: 0.0639
Epoch 5: ReduceLROnPlateau reducing learning rate to 0.0002500000118743628.
543/543 ━━━━━━━━━━━━━━━━━━━━ 116s 214ms/step - accuracy: 0.9820 - loss